In [1]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, TimestampType
from pyspark.sql.functions import sum, col, count, when, lead, collect_list
from datetime import datetime

In [2]:
spark = SparkSession.builder.master('local[4]').appName('PysparkDataPipeline').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/08/09 19:58:37 WARN Utils: Your hostname, Pramod resolves to a loopback address: 127.0.1.1; using 172.23.91.132 instead (on interface eth0)
23/08/09 19:58:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/09 19:58:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Creating an empty Df -

In [3]:
schema = StructType([StructField('column_a', IntegerType(), True),
                    StructField('column_b', StringType(), False)])

In [65]:
df = spark.createDataFrame([], schema=schema)

In [66]:
df.show()

+--------+--------+
|column_a|column_b|
+--------+--------+
+--------+--------+



#### Append Data into it

In [67]:
df2 = spark.createDataFrame([[1, 'a'], [2, 'b'], [3, 'c'], [None, 'd']], schema=schema)

In [68]:
df2.show()

+--------+--------+
|column_a|column_b|
+--------+--------+
|       1|       a|
|       2|       b|
|       3|       c|
|    null|       d|
+--------+--------+



In [69]:
df = df.union(df2)

In [70]:
df.show()

+--------+--------+
|column_a|column_b|
+--------+--------+
|       1|       a|
|       2|       b|
|       3|       c|
|    null|       d|
+--------+--------+



#### Fill Null Values

In [71]:
df = df.fillna(1, subset=['column_a'])

In [72]:
df.show()

+--------+--------+
|column_a|column_b|
+--------+--------+
|       1|       a|
|       2|       b|
|       3|       c|
|       1|       d|
+--------+--------+



#### Replace all 1's with 2's

In [74]:
from pyspark.sql.functions import when
df = df.replace(1, 2, subset=['column_a'])
df.show()

+--------+--------+
|column_a|column_b|
+--------+--------+
|       2|       a|
|       2|       b|
|       3|       c|
|       2|       d|
+--------+--------+



#### LeetCode SQL problem
#### Replace Employee ID With The Unique Identifier
https://leetcode.com/problems/replace-employee-id-with-the-unique-identifier/description/?envType=study-plan-v2&envId=top-sql-50

In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
employees_schema = StructType([StructField('id', IntegerType(), False), StructField('name', StringType(), True)])
employees_df = spark.createDataFrame([[1, 'Alice'], [7, 'Bob'], [11, 'Meir'], [90, 'Winston'], [3, 'Jonathan']], schema=employees_schema)

employee_uni_schema = StructType([StructField('id', IntegerType(), False), StructField('unique_id', IntegerType(), True)])
employee_uni_df = spark.createDataFrame([[3, 1], [11, 2], [90, 3]], schema=employee_uni_schema)

In [10]:
employees_df.show()

+---+--------+
| id|    name|
+---+--------+
|  1|   Alice|
|  7|     Bob|
| 11|    Meir|
| 90| Winston|
|  3|Jonathan|
+---+--------+



In [11]:
employee_uni_df.show()

+---+---------+
| id|unique_id|
+---+---------+
|  3|        1|
| 11|        2|
| 90|        3|
+---+---------+



In [16]:
employees_df.join(employee_uni_df, how='left', on=['id']).select('unique_id', 'name').show()

+---------+--------+
|unique_id|    name|
+---------+--------+
|     null|   Alice|
|     null|     Bob|
|        2|    Meir|
|        1|Jonathan|
|        3| Winston|
+---------+--------+



#### Leetcode SQL problem - Confirmation Rate
https://leetcode.com/problems/confirmation-rate/description/?envType=study-plan-v2&envId=top-sql-50

In [5]:
signups_schema = StructType([StructField('user_id', IntegerType(), False),
                             StructField('time_stamp', TimestampType(), True)])
confirmations_schema = StructType([StructField('user_id', IntegerType(), True),
                                   StructField('time_stamp', TimestampType(), True),
                                   StructField('action', StringType(), True)])

In [11]:
sinups_df = spark.createDataFrame([[3, datetime.strptime('2020-03-21 10:16:13', '%Y-%m-%d %H:%M:%S')], [7, datetime.strptime('2020-01-04 13:57:59', '%Y-%m-%d %H:%M:%S')], [
                                  2, datetime.strptime('2020-07-29 23:09:44', '%Y-%m-%d %H:%M:%S')], [6, datetime.strptime('2020-12-09 10:39:37', '%Y-%m-%d %H:%M:%S')]], schema=signups_schema)
confirmations_df = spark.createDataFrame([[3, datetime.strptime('2021-01-06 03:30:46', '%Y-%m-%d %H:%M:%S'), 'timeout'], [3, datetime.strptime('2021-07-14 14:00:00', '%Y-%m-%d %H:%M:%S'), 'timeout'], [7, datetime.strptime('2021-06-12 11:57:29', '%Y-%m-%d %H:%M:%S'), 'confirmed'], [7, datetime.strptime('2021-06-13 12:58:28', '%Y-%m-%d %H:%M:%S'), 'confirmed'], [
                                         7, datetime.strptime('2021-06-14 13:59:27', '%Y-%m-%d %H:%M:%S'), 'confirmed'], [2, datetime.strptime('2021-01-22 00:00:00', '%Y-%m-%d %H:%M:%S'), 'confirmed'], [2, datetime.strptime('2021-02-28 23:59:59', '%Y-%m-%d %H:%M:%S'), 'timeout']], schema=confirmations_schema)

In [17]:
result_df = sinups_df.join(confirmations_df, how='left', on=['user_id'])

In [50]:
result_df.groupBy('user_id').agg((sum(when(result_df.action == 'confirmed', 1).otherwise(0.00))/ count('*')).alias('confirmation_rate')).show()

+-------+-----------------+
|user_id|confirmation_rate|
+-------+-----------------+
|      3|              0.0|
|      7|              1.0|
|      2|              0.5|
|      6|              0.0|
+-------+-----------------+



#### Find Users With Valid E-Mails
##### https://leetcode.com/problems/find-users-with-valid-e-mails/description/?envType=study-plan-v2&envId=30-days-of-pandas&lang=pythondata

In [5]:
schema = StructType([StructField('user_id', IntegerType(), False), StructField('name', StringType(), True), StructField('mail', StringType(), True)])
df = spark.createDataFrame([[1, 'Winston', 'winston@leetcode.com'],
                            [2, 'Jonathan', 'jonathanisgreat'],
                            [3, 'Annabelle', 'bella-@leetcode.com'],
                            [4, 'Sally', 'sally.come@leetcode.com'],
                            [5, 'Marwan', 'quarz#2020@leetcode.com'],
                            [6, 'David', 'david69@gmail.com'],
                            [7, 'Shapiro', '.shapo@leetcode.com']], schema=schema)

In [6]:
df.show()

+-------+---------+--------------------+
|user_id|     name|                mail|
+-------+---------+--------------------+
|      1|  Winston|winston@leetcode.com|
|      2| Jonathan|     jonathanisgreat|
|      3|Annabelle| bella-@leetcode.com|
|      4|    Sally|sally.come@leetco...|
|      5|   Marwan|quarz#2020@leetco...|
|      6|    David|   david69@gmail.com|
|      7|  Shapiro| .shapo@leetcode.com|
+-------+---------+--------------------+



In [7]:
df.filter(col('mail').rlike('^[a-zA-Z][a-zA-Z0-9_.-]*@leetcode\.com')).show()

+-------+---------+--------------------+
|user_id|     name|                mail|
+-------+---------+--------------------+
|      1|  Winston|winston@leetcode.com|
|      3|Annabelle| bella-@leetcode.com|
|      4|    Sally|sally.come@leetco...|
+-------+---------+--------------------+



#### Consecutive Numbers
#### https://leetcode.com/problems/consecutive-numbers/

In [33]:
schema = StructType([StructField('id', IntegerType(), False), StructField('num', StringType(), True)])
df = spark.createDataFrame([[1, '1'], [2, '1'], [3, '1'], [4, '2'], [5, '1'], [6, '2'], [7, '2']], schema=schema)

In [10]:
df.createOrReplaceTempView('df')

In [16]:
spark.sql('''with temp as (
                select num,
                    lead(num, 1) over (order by id) num1,
                    lead(num, 2) over (order by id) num2 
                from df
                )
                select distinct num 
                from temp
                where num = num1
                and num = num2''').show()

+---+
|num|
+---+
|  1|
+---+



23/08/03 15:05:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/03 15:05:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/03 15:05:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/03 15:05:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [48]:
df.withColumn('num1', lead('num', 1).over(Window.orderBy('id')))\
    .withColumn('num2', lead('num', 2).over(Window.orderBy('id')))\
    .filter((col('num') == col('num1')) & (col('num') == col('num2')))\
    .select('id').distinct().show()

+---+
| id|
+---+
|  1|
+---+



23/08/03 15:56:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/03 15:56:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/03 15:56:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/03 15:56:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


### Students and classes example

In [54]:
schema = StructType([StructField('class', IntegerType(), True),
                    StructField('student_id', IntegerType(), True),
                    StructField('term', IntegerType(), True),
                    StructField('subject', StringType(), True),
                    StructField('marks', IntegerType(), True)])

df = spark.createDataFrame([[2, 1, 1, 'maths', 10], [2, 1, 2, 'maths', 12], [2, 1, 1, 'english', 14], [2, 1, 2, 'english', 12],
                            [3, 2, 1, 'maths', 10], [3, 2, 2, 'maths', 12], [3, 2, 1, 'english', 16], [3, 2, 2, 'english', 14]], schema=schema)

In [55]:
df.show()

+-----+----------+----+-------+-----+
|class|student_id|term|subject|marks|
+-----+----------+----+-------+-----+
|    2|         1|   1|  maths|   10|
|    2|         1|   2|  maths|   12|
|    2|         1|   1|english|   14|
|    2|         1|   2|english|   12|
|    3|         2|   1|  maths|   10|
|    3|         2|   2|  maths|   12|
|    3|         2|   1|english|   16|
|    3|         2|   2|english|   14|
+-----+----------+----+-------+-----+



> Get class 2 students in following format --> class student_id subject term1 term2

In [56]:
df = df.orderBy('class', 'student_id', 'subject')\
    .groupBy('class', 'student_id', 'subject')\
    .agg(collect_list('marks')[0].alias('term1'), collect_list('marks')[1].alias('term2'))

In [57]:
df.show()

+-----+----------+-------+-----+-----+
|class|student_id|subject|term1|term2|
+-----+----------+-------+-----+-----+
|    2|         1|english|   14|   12|
|    2|         1|  maths|   10|   12|
|    3|         2|english|   16|   14|
|    3|         2|  maths|   10|   12|
+-----+----------+-------+-----+-----+



In [58]:
df.filter(col('class') == 2).show()

+-----+----------+-------+-----+-----+
|class|student_id|subject|term1|term2|
+-----+----------+-------+-----+-----+
|    2|         1|english|   14|   12|
|    2|         1|  maths|   10|   12|
+-----+----------+-------+-----+-----+



> Get subject-wise aggregated score with 25% weightage to term1 and 75% weightage to term2

In [67]:
df = spark.createDataFrame([[2, 1, 1, 'maths', 10], [2, 1, 2, 'maths', 12], [2, 1, 1, 'english', 14], [2, 1, 2, 'english', 12],
                            [3, 2, 1, 'maths', 10], [3, 2, 2, 'maths', 12], [3, 2, 1, 'english', 16], [3, 2, 2, 'english', 14]], schema=schema)

In [68]:
df.show()

+-----+----------+----+-------+-----+
|class|student_id|term|subject|marks|
+-----+----------+----+-------+-----+
|    2|         1|   1|  maths|   10|
|    2|         1|   2|  maths|   12|
|    2|         1|   1|english|   14|
|    2|         1|   2|english|   12|
|    3|         2|   1|  maths|   10|
|    3|         2|   2|  maths|   12|
|    3|         2|   1|english|   16|
|    3|         2|   2|english|   14|
+-----+----------+----+-------+-----+



In [69]:
maths_agg = \
    df.filter(col('subject') == 'maths')\
    .orderBy('student_id', 'term')\
    .groupBy('student_id')\
    .agg((collect_list('marks')[0] * 0.25 + collect_list('marks')[1] * 0.75).alias('maths_agg'))

In [70]:
english_agg = \
    df.filter(col('subject') == 'english')\
    .orderBy('student_id', 'term')\
    .groupBy('student_id')\
    .agg((collect_list('marks')[0] * 0.25 + collect_list('marks')[1] * 0.75).alias('english_agg'))

In [71]:
maths_agg.join(english_agg, how='inner', on='student_id').show()

+----------+---------+-----------+
|student_id|maths_agg|english_agg|
+----------+---------+-----------+
|         1|     11.5|       12.5|
|         2|     11.5|       14.5|
+----------+---------+-----------+



#### Exchange Seats -
#### https://leetcode.com/problems/exchange-seats/description/

In [3]:
import pandas as pd
import io
data = '''
id,student
1,Abbot
2,Doris
3,Emerson
4,Green
5,Jeames
'''
df = spark.createDataFrame(pd.read_csv(io.StringIO(data), header=0))

In [4]:
df.show()

+---+-------+
| id|student|
+---+-------+
|  1|  Abbot|
|  2|  Doris|
|  3|Emerson|
|  4|  Green|
|  5| Jeames|
+---+-------+



In [33]:
max_id = max(df.select('id').collect())[0]
df.select(when(col('id') % 2 == 1,
               lead('id', 1, max_id).over(Window.orderBy('id')))
          .otherwise(col('id') - 1).alias('id'),
          'Student').orderBy('id').show()

+---+-------+
| id|Student|
+---+-------+
|  1|  Doris|
|  2|  Abbot|
|  3|  Green|
|  4|Emerson|
|  5| Jeames|
+---+-------+



23/08/09 17:41:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/09 17:41:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/09 17:41:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/09 17:41:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/09 17:41:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### Tree Node
#### https://leetcode.com/problems/tree-node/

In [3]:
schema = StructType([StructField('id', IntegerType(), False), StructField('p_id', IntegerType(), True)])
df = spark.createDataFrame([[1, None], [2, 1], [3, 1], [4, 2], [5, 2]], schema=schema)

In [4]:
df.show()

+---+----+
| id|p_id|
+---+----+
|  1|null|
|  2|   1|
|  3|   1|
|  4|   2|
|  5|   2|
+---+----+



In [18]:
distinct_parent_ids = df.select('p_id').distinct().rdd.flatMap(lambda x: x).collect()
df.select('id', when(col('p_id').isNull(), 'Root')
          .when(col('p_id').isNotNull() & col('id').isin(distinct_parent_ids), 'Inner')
          .otherwise('Leaf')).show()

+---+--------------------------------------------------------------------------------------------------------------+
| id|CASE WHEN (p_id IS NULL) THEN Root WHEN ((p_id IS NOT NULL) AND (id IN (NULL, 1, 2))) THEN Inner ELSE Leaf END|
+---+--------------------------------------------------------------------------------------------------------------+
|  1|                                                                                                          Root|
|  2|                                                                                                         Inner|
|  3|                                                                                                          Leaf|
|  4|                                                                                                          Leaf|
|  5|                                                                                                          Leaf|
+---+-----------------------------------------------------------

#### Pandas

In [51]:
pdf = df.toPandas()

In [52]:
pdf

,id,p_id
0,1,NaN
1,2,1.0
2,3,1.0
3,4,2.0
4,5,2.0


In [54]:
import numpy as np
pdf['type'] = np.where(pdf['p_id'].isna(),
                       'Root',
                       np.where(pdf['id'].isin(pdf['p_id'].unique()) & pdf['p_id'].notna(),
                                'Inner',
                                'Leaf'))

In [55]:
pdf[['id', 'type']]

,id,type
0,1,Root
1,2,Inner
2,3,Leaf
3,4,Leaf
4,5,Leaf
